In [39]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mpmath import nsum, exp, inf
from main import get_all_retention_rates_for_cohort, get_data

# 4a

- $p_{c_k}$ : Active probability from constant retention rate. $p_{c_1} = 1$
- $r_c$ : Constant Retention rate

$$\mathbb{E}_c [user \space months] = \sum_{k=1}^{\inf}k \cdot p_{c_{k}} = \sum_{k=1}^{\inf}k \cdot r_c^{k-1}$$

In [40]:
data = get_data()

constant_retention_rate = 0.56530017

active_probs_constant_retention = [constant_retention_rate**i for i in range(1000)]
#E[lifetime] = 1*retention_rate + 2*retention_rate^2 +..n*retention_rate^3
# print(active_probs_constant_retention)
# print(nsum(lambda n: n*active_probs_constant_retention[n-1], [1, 100]))
expected_value_constant_retention = sum([i*active_probs_constant_retention[i-1] for i in range(1,len(active_probs_constant_retention))])
print(expected_value_constant_retention)

5.29201491219346


# 4b

**OLD:**


*Should develop strategies to keep the customers in the second month*


*The expected lifetime of above 2 is heavily influenced by the loyal customers, that have been staying for many months and therefore driving the expected value up. On the other side we have seen big drops of customers after only one or two months, and we would therefore want to get more users past that critical point of two months. This could be done by more effectively showing the customer relevant content. It is however difficult to personalize content for a user that they have little data on, and one strategy could therefore be to gather data about the particular user from outside of Netflix itself to propose relevant, personalized content that may make the user stay past the critical point*

$5.29$ is a rather high expected lifetime for customers. This is using the constant retention rate and assuming everyone finishes at least one month. The math looks something like $1 + 2\cdot 0.57^1 + 3 \cdot 0.57^2 + 4 \cdot 0.57^3  \cdots = 1 + 1.14 + 0.97 + 0.74 \cdots $. We have already discussed that the average retention rate is artificially high, and so this expected value seems high as well as a consequence. It makes however sence if we consider the long-serving customers and not only the ones acquired in our time window of data. And so this expected value rather represents one type of the general customer base which are the long-serving ones

# 4c

In [47]:

def get_all_retention_rates_time_normalized(data):
    retention_rates_all_cohorts = []
    for i in range(12):
        all_retention_rates_for_cohort = (get_all_retention_rates_for_cohort(data,i))
        retention_rates_all_cohorts.append(all_retention_rates_for_cohort)
    for j in range(1,len(retention_rates_all_cohorts)):
        for i in range(j,len(retention_rates_all_cohorts[j])):
            retention_rates_all_cohorts[j][i-j] = retention_rates_all_cohorts[j][i]
            retention_rates_all_cohorts[j][i] = np.NaN
    return retention_rates_all_cohorts

get_all_retention_rates_months_after_acquisition = get_all_retention_rates_time_normalized(data)

avg_monthly_retention_rate_across_cohorts = np.apply_along_axis(np.nanmean,axis = 0, arr = get_all_retention_rates_months_after_acquisition)
print(avg_monthly_retention_rate_across_cohorts)

active_probs_changing_retention = avg_monthly_retention_rate_across_cohorts[1:-1]
active_probs_changing_retention = np.insert(active_probs_changing_retention,0,1)
active_probs_changing_retention = np.cumprod(active_probs_changing_retention)
# active_probs_changing_retention = np.append(active_probs_changing_retention,np.NaN)
print(active_probs_changing_retention)

expected_value = 0
i=1
for prob in list(active_probs_changing_retention):
    expected_value += prob*i
    i+=1

print(expected_value)


[       nan 0.48308715 0.11909775 0.24597981 0.42569815 0.44580342
 0.6        0.33333333 1.         1.         1.                nan]
[1.00000000e+00 4.83087149e-01 5.75345930e-02 1.41523483e-02
 6.02462849e-03 2.68580000e-03 1.61148000e-03 5.37160000e-04
 5.37160000e-04 5.37160000e-04 5.37160000e-04]
2.273317852896713


In [48]:
expectation_value = np.arange(1,len(active_probs_changing_retention)+1,1)@active_probs_changing_retention
expectation_value

2.273317852896713

# 4d

The values for the different methods of calculating are radically different. Similarly as before we see here a sort of "abuse" of doing the average of averages in eliminating the relevance of the sample size. In this case in 4a we are using the average retention rate of the average retention rates of each month after customer acquisition.
To get the relevance of the newly acquired customers, also the ones that churned early, the last retention rate in 4c should be used as it is more realistic.
Using this, we realize that many churn after only one month, and the active probabilities fall quicker then in the first case

In [43]:
#4d
#Strategies for keeping customers during first month vs second month